## Scrapy的下载中间件详解
　　参考：[官方文档](https://docs.scrapy.org/en/2.0/topics/settings.html?highlight=DOWNLOADER_MIDDLEWARES#downloader-middlewares-base)<br>
　　　　　[Scrapy爬虫之下载中间件的处理流程](https://blog.csdn.net/qq_35556064/article/details/91346493)<br>
　　　　　[一些常用中间件编写](https://www.jianshu.com/p/a91c663579eb)<br>


### 一、Download Middleware（下载中间件）使用
　　Download Middleware是Scrapy的请求/响应处理的钩子框架。它是一个轻量级的低级系统，用于全局改变Scrapy的请求和响应。常用来添加代理，添加cookie，失败重新发起请求等等。<br>

　　#### 1、激活Download Middleware
　　要激活Download Middleware，请在settings.py中激活 **DOWNLOADER_MIDDLEWARES**设置，该设置是一个dict，其键是中间件类路径，其值是中间件处理顺序值。要禁用某个中间件，只需把值设为None即可。<br>
　　这是一个例子：
>DOWNLOADER_MIDDLEWARES = {<br>
    'myproject.middlewares.CustomDownloaderMiddleware': 543,<br>
}<br>

在之前的代码运行时，我们在日志可以看到很多downloadermiddlewares默认被配置启动，这些是通过DOWNLOADER_MIDDLEWARES_BASE配置的。<br>

>2019-04-07 15:28:48 [scrapy.middleware] INFO: Enabled downloader middlewares:<br>
['scrapy.downloadermiddlewares.robotstxt.RobotsTxtMiddleware',<br>
 'scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',<br>
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',<br>
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',<br>
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',<br>
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',<br>
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',<br>
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',<br>
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',<br>
 'scrapy.downloadermiddlewares.cookies.CookiesMiddleware',<br>
 'scrapy.downloadermiddlewares.stats.DownloaderStats']<br>
2019-04-07 15:28:48 [scrapy.middleware] INFO: Enabled spider middlewares:<br>
['scrapy.spidermiddlewares.httperror.HttpErrorMiddleware',<br>
 'scrapy.spidermiddlewares.offsite.OffsiteMiddleware',<br>
 'scrapy.spidermiddlewares.referer.RefererMiddleware',<br>
 'scrapy.spidermiddlewares.urllength.UrlLengthMiddleware',<br>
 'scrapy.spidermiddlewares.depth.DepthMiddleware']<br>

　　如果要禁用内置中间件DOWNLOADER_MIDDLEWARES_BASE默认定义和启用的中间件 ，则必须在项目的DOWNLOADER_MIDDLEWARES设置中定义它为None。例如，如果要禁用用户代理中间件：<br>
>DOWNLOADER_MIDDLEWARES = {<br>
    'myproject.middlewares.CustomDownloaderMiddleware': 543,<br>
    'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware': None,<br>
}

　　#### 2、编写定义自己的下载中间件
　　每个中间件组件都是一个Python类，它定义了以下一种或多种方法：<br>

##### 1、process_request（request, spider）
对于通过Download Middleware的每个请求，都会调用此方法。<br>

process_request()应该：返回None，返回一个 Response对象，返回一个Request对象，或者抛出IgnoreRequest异常。<br>

如果它返回None，Scrapy将继续处理此请求，执行所有其他中间件，直到最后。这个对整个框架没什么影响。<br>

如果它返回一个Response对象，Scrapy将不再调用其他process_request()或process_exception()方法。在每次Response时，已安装的中间件的process_response()方法都会被调用。<br>

如果它返回一个Request对象，Scrapy将停止调用process_request方法并重新安排返回的请求。从而循环往复地调度Request。<br>

如果它引发IgnoreRequest异常，已安装的下载中间件的process_exception()方法将被调用。如果它们都不处理异常，则调用request()(Request.errback)中的errback函数。如果没有代码处理引发的异常，则会忽略它并且不会记录。<br>

参数：<br>

request（Request对象） - 正在处理的请求<br>
spider（Spider对象） - 此请求所针对的蜘蛛<br>
##### 2、process_response (request，response，spider)

process_response()应该：返回一个Response对象，返回一个Request对象或抛出IgnoreRequest异常。<br>

如果它返回 Response，将继续处理下一个中间件的process_response()。也就是对其他中间件没影响。<br>

如果它返回一个Request对象，就不会调用process_response()，而是将process_request()重新加入到调度队列。<br>

如果它引发IgnoreRequest异常，已安装的下载中间件的process_exception()方法将被调用。如果它们都不处理异常，则调用request()(Request.errback)中的errback函数。如果没有代码处理引发的异常，则会忽略它并且不会记录。<br>

参数：<br>

request（Request对象） - 发起响应的请求<br>
response（Response对象） - 正在处理的响应<br>
spider（Spider对象） - 此响应所针对的蜘蛛<br>
##### 3、process_exception(requset, exception, spider)

process_exception()应该返回：要么None是Response对象，要么是Request对象。<br>

如果它返回None，Scrapy将继续处理此异常，执行任何其他已安装中间件的process_exception()方法，直到没有剩下中间件并且默认异常处理开始。<br>

如果它返回一个Response对象，process_response()则启动已安装的中间件的方法链，并且Scrapy不会调用任何其他process_exception()中间件方法。<br>

如果它返回一个Request对象，则重新安排返回的请求以便将来下载。这会停止执行process_exception()中间件的方法，就像返回响应一样。这个用于失败重复调用时很有用。<br>

参数：<br>

request（是一个Request对象） - 生成异常的请求<br>
exception（一个Exception对象） - 引发的异常<br>
spider（Spider对象） - 此请求所针对的蜘蛛<br>


### 二、源码解析


### 三、原理解析
![下载中间件数据处理流程图](./images/scrapy_downloadermidder1.png)<br><br>
Scrapy的下载中间件Download Middleware有三个函数核心处理函数(其数据处理传递流程如上图)：

|函数|返回对象|处理流程|
|:----|:-----|:----|
|process_request|1.None<br>2.Response<br>3.Request<br>4.Raise IgnoreRequest|1->函数返回None的处理流程<br>2->返回Response对象的处理流程<br>3->返回Request的处理流程<br>4->返回Exception的处理流程|
|process_response|A.Response<br>B.Request<br>C.Raise IgnoreRequest|A->返回Response对象的处理流程<br>B->返回Request的处理流程<br>C->返回Exception的处理流程|
|process_exception|i.None<br>ii.Response<br>ii.Request|i->返回None的处理流程<br>ii->返回Response的处理流程<br>iii->返回Request的处理流程|

#### 1、process_request(self, request, spider)
以图中的下载中间件2为例说明：
>返回None：将请求传给后面的中间件的process_request函数继续处理<br>
返回Response：请求不再传给后面的中间件，而是直接进入最后一个中间件的process_response函数里来处理Response对象<br>
返回Request：重新开始处理请求，即进入第一个中间件的process_request函数<br>
Raise IgnoreRequest：断言请求忽略的异常，直接进入最后一个中间件的process_exception函数，用该函数来处理这个异常；如果没有任何中间件的此函数来处理这个异常，则再调用Reuest对象的errorblack回调函数来处理；如果连这个回调都没有，则引擎直接忽略该异常。<br>
#### 2、process_response(self, request, response,spider)
以图中的下载中间件3为例说明：
>返回Response：将响应交由下一个中间件的process_response函数继续处理<br>
返回Request：请求将重新开始，即进入第一个中间件的process_request函数<br>
Raise IgnoreRequest：注意此时不会进入process_exception函数，而是调用Reuest对象的errorblack回调函数来处理，如果没有定义这个回调则忽略。<br>
#### 3、process_exception(self, request, exception, spider)
以图中的下载中间件3为例说明：
>返回None：将异常交给下一个中间件的process_exception函数处理<br>
返回Response： 将该响应交由最后一个中间件的process_response函数处理<br>
返回Request：重新开始处理请求，即进入第一个中间件的process_request函数<br>

**备注：**<br>
中间件需要安装才能处理，请记得安装配置中间件的优先级。<br>
中间件的优先级：数字越小越靠近引擎，数字越大越靠近下载器。请求最先从引擎处开始传递，逐次经过优先级数字越来越大的中间件；相反，下载器返回响应时，则逐次经过优先级数字越来越小的中间件处理。因此，以上图为例：不要认为一个请求最先是中间件1处理的，那么响应也会最先由中间件1来处理，这是错误的。请求最先由优先级数字最小的中间件来处理，而响应最先由优先级数字最大的中间件处理<br>

